In [17]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
from statsmodels.stats.multitest import multipletests

In [18]:
all_mean_arr=-0.017462454026226405
all_std_arr=0.006996042091888453
mean_dict_different_race={'ACB': -0.021129940274677197,
 'ASW': -0.020541364517036285,
 'BEB': -0.01690678177281044,
 'CDX': -0.01520099465128729,
 'CEU': -0.016533114461341817,
 'CHB': -0.015328450587149127,
 'CHS': -0.01553630769670323,
 'CLM': -0.017175762377800757,
 'ESN': -0.021289481956977087,
 'FIN': -0.016220012008386193,
 'GBR': -0.01616972796402513,
 'GIH': -0.016647828395962485,
 'GWD': -0.02127146261125899,
 'IBS': -0.016612028858784364,
 'ITU': -0.016483044568386403,
 'JPT': -0.015297644767223825,
 'KHV': -0.015510621661266998,
 'LWK': -0.020977118301868332,
 'MSL': -0.021221971390719675,
 'MXL': -0.016358508148574633,
 'PEL': -0.015192057641423692,
 'PJL': -0.016650973867877568,
 'PUR': -0.01772660210939756,
 'STU': -0.016488613724290634,
 'TSI': -0.016378152064038587,
 'YRI': -0.021395734138643974}
std_dict_different_race={'ACB': 0.007866911384050993,
 'ASW': 0.007845222560823078,
 'BEB': 0.007884710711424594,
 'CDX': 0.007874313523803581,
 'CEU': 0.007927585629080875,
 'CHB': 0.007946756589325447,
 'CHS': 0.00794164324551959,
 'CLM': 0.007812970518334978,
 'ESN': 0.00785082325216948,
 'FIN': 0.007906254940044924,
 'GBR': 0.007932404777081037,
 'GIH': 0.007873289737229847,
 'GWD': 0.007853536883558044,
 'IBS': 0.007919548085211019,
 'ITU': 0.007768497466178242,
 'JPT': 0.00796914647357699,
 'KHV': 0.007990414654274063,
 'LWK': 0.007797858211516251,
 'MSL': 0.007864701129783158,
 'MXL': 0.007798067190318037,
 'PEL': 0.007722619015518075,
 'PJL': 0.007712973460516593,
 'PUR': 0.007785891217808583,
 'STU': 0.0077724707332740475,
 'TSI': 0.00791282517960449,
 'YRI': 0.007915268495722334}

In [19]:
mhc_race_positions_dict={'ACB': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'All':{'mhc':[[29720403, 30048796],[30994370, 31528792],[32212726, 32882258]]},
 'ASW': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'BEB': {'mhc': [[29720403, 30048796],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CDX': {'mhc': [[30994370, 31528792], [32212726, 32923168]]},
 'CEU': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'CHB': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'CHS': {'mhc': [[30994370, 31528792], [32397207, 32882258]]},
 'CLM': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ESN': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32397207, 32882258]]},
 'FIN': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GBR': {'mhc': [[30994370, 31528792], [32212726, 32882258]]},
 'GIH': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'GWD': {'mhc': [[29720403, 30120966],
   [31052133, 31528792],
   [32212726, 32882258]]},
 'IBS': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'ITU': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32923168]]},
 'JPT': {'mhc': [[29720403, 29913914],
   [29939668, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'KHV': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32397207, 32923168]]},
 'LWK': {'mhc': [[29720403, 30011739],
   [31052133, 31528792],
   [32212726, 32923168]]},
 'MSL': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'MXL': {'mhc': [[29720403, 29913914],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'PEL': {'mhc': [[29720403, 29896285],
   [30959575, 31528792],
   [32288923, 32882258]]},
 'PJL': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32923168]]},
 'PUR': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'STU': {'mhc': [[29720403, 30011739],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'TSI': {'mhc': [[29720403, 29896285],
   [30994370, 31528792],
   [32212726, 32882258]]},
 'YRI': {'mhc': [[29720403, 30120966],
   [30994370, 31528792],
   [32212726, 32882258]]}}

In [20]:
split_race_region={'ACB': {},
 'ASW': {},
 'BEB': {'split_chr3/xav': [[98046342, 98273693]]},
 'CDX': {'split_chr13/xai': [[55102733, 55443491]]},
 'CEU': {'split_chr17/xaj': [[45670177, 46418024]]},
 'CHB': {},
 'CHS': {'split_chr3/xav': [[98046342, 98273693]]},
 'CLM': {'split_chr17/xaj': [[45836662, 46224960]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'ESN': {'split_chr5/xai': [[34264948, 34562837]]},
 'FIN': {},
 'GBR': {'split_chr1/xbt': [[226978066, 227352188]],
  'split_chr17/xaj': [[45670177, 46418024]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'GIH': {'split_chr3/xav': [[98046342, 98276757]]},
 'GWD': {},
 'IBS': {'split_chr17/xaj': [[45670177, 46418024]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'ITU': {'split_chr17/xak': [[52575391, 52889013]]},
 'JPT': {},
 'KHV': {},
 'LWK': {},
 'MSL': {'split_chr5/xai': [[34264948, 34562837]]},
 'MXL': {'split_chr1/xbt': [[227016487, 227352188]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'PEL': {'split_chr11/xaf': [[23168664, 23564494]],
  'split_chr3/xav': [[98046342, 98276757]]},
 'PJL': {},
 'PUR': {'split_chr17/xaj': [[45836662, 46418024]],
  'split_chr3/xav': [[98046342, 98276757]]},
 'STU': {'split_chr13/xai': [[55102733, 55443491]],
  'split_chr3/xav': [[98046342, 98273693]]},
 'TSI': {'split_chr17/xaj': [[45670177, 46418024]]},
 'YRI': {}}

In [21]:
import re
def getinteger(s):
    match = re.search(r'chr(\d+)', s)

    if match:
        return match.group(1) # 输出: 7


In [22]:
df_mhc_all=pd.read_csv("../../similar_region_csvfiles/posmhc.csv")
df_mhc_race=pd.read_csv("../../similar_region_race/mhc.csv")

In [23]:
allrecords=[]
for race,values in mhc_race_positions_dict.items():
    if values!={}:
        if race=="All":
            for region in values["mhc"]:
                onerecord=[]
                onerecord+=["All",6,1]
                top_1000_P_list=df_mhc_all[(df_mhc_all["Unnamed: 0"]>=region[0])&(df_mhc_all["Unnamed: 0"]<=region[1])]["P"].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round(stats.norm.cdf(mean_1000_P, all_mean_arr, all_std_arr),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)
        else:
            for region in values["mhc"]:
                onerecord=[]
                onerecord+=[race,6,1]
                top_1000_P_list=df_mhc_race[(df_mhc_race["Unnamed: 0"]>=region[0])&(df_mhc_race["Unnamed: 0"]<=region[1])][race].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round(stats.norm.cdf(mean_1000_P, mean_dict_different_race[race], std_dict_different_race[race]),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)


In [24]:
for race,values in split_race_region.items():

    for k,v in values.items():
        chr=getinteger(k.split("/")[0])
        filename=k.split("/")[0]+"_"+k.split("/")[1]+".csv"
        df=pd.read_csv("../../similar_region_race/"+filename)
        for region in v:
            onerecord=[]
            onerecord+=[race,chr,0]
            top_1000_P_list=df[(df["Unnamed: 0"]>=region[0])&(df["Unnamed: 0"]<=region[1])][race].to_list()
            # sortedlist=sorted(top_1000_P_list,reverse=True)
            # top_1000=sortedlist[:1000]
            mean_1000_P=np.mean(top_1000_P_list)
            std_1000_P=np.std(top_1000_P_list, ddof=1)
            pvalue=round(stats.norm.cdf(mean_1000_P,mean_dict_different_race[race], std_dict_different_race[race]),15)
            onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
            allrecords.append(onerecord)

In [25]:
#csv1
columns=["Race","chr","isMHC","Average of P","Std of P","P-value","start","end","len"]

In [26]:
#generatedf
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    # dataframe(dictforDF).to_csv("../formalcsvs/csv1_2023_05_04.csv")
    return dataframe(dictforDF)





In [27]:
dfnew=generatedf(columns=columns,allrecords=allrecords)

In [28]:
dfnew

,Race,chr,isMHC,Average of P,Std of P,P-value,start,end,len
0,ACB,6,1,-0.066278,0.076288,4.761017e-09,29720403,30011739,291336
1,ACB,6,1,-0.060594,0.081530,2.631978e-07,30994370,31528792,534422
2,ACB,6,1,-0.082833,0.099343,2.000000e-15,32212726,32882258,669532
3,All,6,1,-0.056621,0.070783,1.088514e-08,29720403,30048796,328393
4,All,6,1,-0.067494,0.082917,4.300000e-13,30994370,31528792,534422
...,...,...,...,...,...,...,...,...,...
97,PUR,17,0,-0.037595,0.064749,5.358631e-03,45836662,46418024,581362
98,PUR,3,0,-0.049351,0.094927,2.435888e-05,98046342,98276757,230415
99,STU,13,0,-0.036510,0.090015,4.999040e-03,55102733,55443491,340758
100,STU,3,0,-0.053836,0.109779,7.734840e-07,98046342,98273693,227351


In [29]:
pvalue_list=dfnew["P-value"].to_list()
pvalue_list

[4.761017e-09,
 2.63197754e-07,
 2e-15,
 1.0885136e-08,
 4.3e-13,
 0.0,
 2.2044e-11,
 1.0485966e-08,
 2.7918e-11,
 8e-15,
 5.312884e-09,
 6.33796e-10,
 1.25e-13,
 3.11839e-10,
 7.099e-12,
 2.50206e-10,
 1e-15,
 8.9118831e-08,
 5.6456e-11,
 6.9e-14,
 1.052914e-08,
 1e-15,
 1.39095913e-07,
 2.405e-12,
 0.0,
 5.748873e-09,
 3.3071383e-08,
 2.66e-13,
 7.574e-12,
 4.37e-13,
 1.969359e-09,
 0.0,
 3.77832887e-07,
 3.765e-12,
 0.0,
 3.407e-11,
 7.40732e-10,
 3.109e-12,
 1.053741333e-06,
 7.454e-12,
 0.0,
 8.2884923e-08,
 9.36304e-10,
 0.0,
 3.084744e-09,
 3.76585391e-07,
 6.765436e-09,
 0.0,
 2.598375e-09,
 1.58247e-10,
 0.0,
 1.748679e-09,
 6.44945e-10,
 1.5785e-11,
 2.25e-13,
 2.77495e-09,
 4.7e-14,
 1.0741595e-08,
 1.2828e-11,
 0.0,
 4.714798197e-06,
 8.379e-12,
 7.6e-13,
 1.90061107e-07,
 4.7292e-11,
 1.01e-13,
 5.894605e-09,
 3.65e-12,
 4e-15,
 2.1405064e-08,
 2.631e-12,
 0.0,
 7.28636181e-07,
 4.27437e-10,
 0.0,
 1.239742e-09,
 6.018933e-09,
 2.6e-14,
 6.7476493378e-05,
 0.00587148059895

In [30]:
pvalue_list=dfnew["P-value"].to_list()
corr_pvals = multipletests(pvalue_list, method='fdr_bh')[1]
corr_pvals

array([8.51971463e-09, 3.67755766e-07, 1.45714286e-14, 1.68224829e-08,
       1.78296000e-12, 0.00000000e+00, 5.91707368e-11, 1.67808169e-08,
       7.30163077e-11, 5.10000000e-14, 9.34334772e-09, 1.37050813e-09,
       6.07142857e-13, 7.06835067e-10, 2.26280625e-11, 5.80023000e-10,
       7.84615385e-15, 1.29858868e-07, 1.37107429e-10, 3.70421053e-13,
       1.67808169e-08, 7.84615385e-15, 1.99827931e-07, 9.08555556e-12,
       0.00000000e+00, 9.93872959e-09, 4.96070745e-08, 1.17965217e-12,
       2.27220000e-11, 1.78296000e-12, 3.79008713e-09, 0.00000000e+00,
       5.13852726e-07, 1.23880645e-11, 0.00000000e+00, 8.68785000e-11,
       1.54193192e-09, 1.09351034e-11, 1.37796944e-06, 2.27220000e-11,
       0.00000000e+00, 1.22525538e-07, 1.91006016e-09, 0.00000000e+00,
       5.61864086e-09, 5.13852726e-07, 1.11302334e-08, 0.00000000e+00,
       4.90804167e-09, 3.75376605e-10, 0.00000000e+00, 3.43010112e-09,
       1.37050813e-09, 4.35154054e-11, 1.04318182e-12, 5.14627091e-09,
      

In [31]:
dfnew = dfnew.assign(corr_pvals=corr_pvals)

In [32]:
dfnew

,Race,chr,isMHC,Average of P,Std of P,P-value,start,end,len,corr_pvals
0,ACB,6,1,-0.066278,0.076288,4.761017e-09,29720403,30011739,291336,8.519715e-09
1,ACB,6,1,-0.060594,0.081530,2.631978e-07,30994370,31528792,534422,3.677558e-07
2,ACB,6,1,-0.082833,0.099343,2.000000e-15,32212726,32882258,669532,1.457143e-14
3,All,6,1,-0.056621,0.070783,1.088514e-08,29720403,30048796,328393,1.682248e-08
4,All,6,1,-0.067494,0.082917,4.300000e-13,30994370,31528792,534422,1.782960e-12
...,...,...,...,...,...,...,...,...,...,...
97,PUR,17,0,-0.037595,0.064749,5.358631e-03,45836662,46418024,581362,5.577350e-03
98,PUR,3,0,-0.049351,0.094927,2.435888e-05,98046342,98276757,230415,2.957864e-05
99,STU,13,0,-0.036510,0.090015,4.999040e-03,55102733,55443491,340758,5.256723e-03
100,STU,3,0,-0.053836,0.109779,7.734840e-07,98046342,98273693,227351,1.024615e-06


In [33]:
dfnew.to_csv("csv1_similar_region.csv")